In [3]:
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import *
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql import Row
from pyspark.sql.functions import *
from pyspark.sql.types import *

# 把调用的package信息写的更详细，避免只用*

In [6]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
spark

## Basic example on Transformer and Estimator

In [7]:
# Prepare training data from a list of (label, features) tuples.
# Dense Vectors are just NumPy arrays

training = spark.createDataFrame([
    (1, Vectors.dense([0.0, 1.1, 0.1])),
    (0, Vectors.dense([2.0, 1.0, -1.0])),
    (0, Vectors.dense([2.0, 1.3, 1.0])),
    (1, Vectors.dense([0.0, 1.2, -0.5]))], ["label", "features"])

# Create a LogisticRegression instance. This instance is an Estimator.
lr = LogisticRegression(maxIter=10, regParam=0.01) # 训练10次，正则化系数0.01

# Print out the parameters, documentation, and any default values.
print (lr.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. The bounds vector size must beequal wi

In [10]:
# Learn a LogisticRegression model. This uses the parameters stored in lr.
model1 = lr.fit(training) # lr是一个estimator，它会返回一个Model

print (model1)
# model1 is a Model (i.e., a transformer produced by an Estimator)
print ("Model 1's trained coefficients: ", model1.coefficients)

LogisticRegressionModel: uid = LogisticRegression_c4e7c61455a1, numClasses = 2, numFeatures = 3
Model 1's trained coefficients:  [-3.1009356010205327,2.60821473832145,-0.38017912254303127]


In [11]:
# We may alternatively specify parameters using a Python dictionary as a paramMap

# 使用python的字典
paramMap = {lr.maxIter: 20}
paramMap[lr.maxIter] = 30  # Specify 1 Param, overwriting the original maxIter.

# 训练30次，正则化系数0.1 - 与model1不一样的地方

paramMap.update({lr.regParam: 0.1, lr.threshold: 0.55})  # Specify multiple Params.

# threshold -- 阈值的意思，但是具体代表什么，有什么作用？待思考

# You can combine paramMaps, which are python dictionaries.
paramMap[lr.probabilityCol] = "myProbability"  # Change output column name

# Now learn a new model using the paramMapCombined parameters.
# paramMapCombined overrides all parameters set earlier via lr.set* methods.
model2 = lr.fit(training, paramMap)
print ("Model 2's trained coefficients: ", model2.coefficients)

Model 2's trained coefficients:  [-1.4313658815706816,0.4320887101487753,-0.1492041947797503]


In [12]:
# Prepare test data
test = spark.createDataFrame([
    (1, Vectors.dense([-1.0, 1.5, 1.3])),
    (2, Vectors.dense([3.0, 2.0, -0.1])),
    (3, Vectors.dense([0.0, 2.2, -1.5]))], ["id", "features"])

# Make predictions on test data using the Transformer.transform() method.
# LogisticRegression.transform will only use the 'features' column.
# Note that model2.transform() outputs a "myProbability" column instead of the usual
# 'probability' column since we renamed the lr.probabilityCol parameter previously.

model1.transform(test).show()
model2.transform(test).show()

+---+--------------+--------------------+--------------------+----------+
| id|      features|       rawPrediction|         probability|prediction|
+---+--------------+--------------------+--------------------+----------+
|  1|[-1.0,1.5,1.3]|[-6.5872014439355...|[0.00137599470692...|       1.0|
|  2|[3.0,2.0,-0.1]|[3.98018281942566...|[0.98166040093741...|       0.0|
|  3|[0.0,2.2,-1.5]|[-6.3765177028604...|[0.00169814755783...|       1.0|
+---+--------------+--------------------+--------------------+----------+

+---+--------------+--------------------+--------------------+----------+
| id|      features|       rawPrediction|       myProbability|prediction|
+---+--------------+--------------------+--------------------+----------+
|  1|[-1.0,1.5,1.3]|[-2.8046569418746...|[0.05707304171033...|       1.0|
|  2|[3.0,2.0,-0.1]|[2.49587635664203...|[0.92385223117040...|       0.0|
|  3|[0.0,2.2,-1.5]|[-2.0935249027914...|[0.10972776114779...|       1.0|
+---+--------------+-----------------

## Pipeline example

In [16]:
# Prepare training documents from a list of (id, text, label) tuples.
training = spark.createDataFrame([
    (0, "A b c d spark spark", 1),
    (1, "b d", 0),
    (2, "SPARK f g h", 1),
    (3, "hadoop mapreduce", 0)
], ["id", "text", "label"])

In [17]:
# A tokenizer converts the input string to lowercase and then splits it by white spaces.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
tokenizer.transform(training).show()

+---+-------------------+-----+--------------------+
| id|               text|label|               words|
+---+-------------------+-----+--------------------+
|  0|A b c d spark spark|    1|[a, b, c, d, spar...|
|  1|                b d|    0|              [b, d]|
|  2|        SPARK f g h|    1|    [spark, f, g, h]|
|  3|   hadoop mapreduce|    0| [hadoop, mapreduce]|
+---+-------------------+-----+--------------------+



In [18]:
# The same can be achieved by DataFrameAPI:
# But you will need to wrap it as a transformer to use it in a pipeline.
# 但是用这个方法不会转换大小写

training.select('*', split(training['text'],' ').alias('words')).show()

+---+-------------------+-----+--------------------+
| id|               text|label|               words|
+---+-------------------+-----+--------------------+
|  0|A b c d spark spark|    1|[A, b, c, d, spar...|
|  1|                b d|    0|              [b, d]|
|  2|        SPARK f g h|    1|    [SPARK, f, g, h]|
|  3|   hadoop mapreduce|    0| [hadoop, mapreduce]|
+---+-------------------+-----+--------------------+



In [20]:
# Maps a sequence of terms to their term frequencies(TF) using the hashing trick.
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
a = hashingTF.transform(tokenizer.transform(training))
a.show(truncate=False)

print (a.select('features').first())

+---+-------------------+-----+--------------------------+----------------------------------------------------------------+
|id |text               |label|words                     |features                                                        |
+---+-------------------+-----+--------------------------+----------------------------------------------------------------+
|0  |A b c d spark spark|1    |[a, b, c, d, spark, spark]|(262144,[27526,28698,30913,227410,234657],[1.0,1.0,1.0,1.0,2.0])|
|1  |b d                |0    |[b, d]                    |(262144,[27526,30913],[1.0,1.0])                                |
|2  |SPARK f g h        |1    |[spark, f, g, h]          |(262144,[15554,24152,51505,234657],[1.0,1.0,1.0,1.0])           |
|3  |hadoop mapreduce   |0    |[hadoop, mapreduce]       |(262144,[42633,155117],[1.0,1.0])                               |
+---+-------------------+-----+--------------------------+----------------------------------------------------------------+

Row(fea

In [21]:
# lr is an estimator
lr = LogisticRegression(maxIter=10, regParam=0.001)

# Now we are ready to assumble the pipeline
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])
                            
# Fit the pipeline to training documents.
model = pipeline.fit(training)

# Prepare test documents, which are unlabeled (id, text) tuples.
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "spark hadoop spark"),
    (7, "apache hadoop")
], ["id", "text"])

# Make predictions on test documents and print columns of interest.
model.transform(test).show()

+---+------------------+--------------------+--------------------+--------------------+--------------------+----------+
| id|              text|               words|            features|       rawPrediction|         probability|prediction|
+---+------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  4|       spark i j k|    [spark, i, j, k]|(262144,[20197,24...|[-0.7500987629693...|[0.32079978124885...|       1.0|
|  5|             l m n|           [l, m, n]|(262144,[18910,10...|[1.58033363406982...|[0.82925176350809...|       0.0|
|  6|spark hadoop spark|[spark, hadoop, s...|(262144,[155117,2...|[-0.7053638244645...|[0.33062407361579...|       1.0|
|  7|     apache hadoop|    [apache, hadoop]|(262144,[66695,15...|[3.95550096961367...|[0.98121072417264...|       0.0|
+---+------------------+--------------------+--------------------+--------------------+--------------------+----------+



--------

## Example: Analyzing food inspection data using logistic regression

In [22]:
# Data at https://www.cse.ust.hk/msbd5003/data/Food_Inspections1.csv

inspections = spark.read.csv('data/Food_Inspections1.csv', inferSchema=True)

Let's take a look at its schema:

In [23]:
inspections.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: double (nullable = true)
 |-- _c15: double (nullable = true)
 |-- _c16: string (nullable = true)



In [24]:
inspections.show()

+------+--------------------+--------------------+-------+--------------------+---------------+--------------------+-------+---+-----+----------+--------------------+------------------+--------------------+------------------+------------------+--------------------+
|   _c0|                 _c1|                 _c2|    _c3|                 _c4|            _c5|                 _c6|    _c7|_c8|  _c9|      _c10|                _c11|              _c12|                _c13|              _c14|              _c15|                _c16|
+------+--------------------+--------------------+-------+--------------------+---------------+--------------------+-------+---+-----+----------+--------------------+------------------+--------------------+------------------+------------------+--------------------+
|413707|       LUNA PARK INC| LUNA PARK  DAY CARE|2049789|Children's Servic...|  Risk 1 (High)|  3250 W FOSTER AVE |CHICAGO| IL|60625|09/21/2010|  License-Task Force|              Fail|24. DISH WASHING 

We now have the CSV file as a DataFrame. It has some columns we will not use. Dropping them can save memory when caching the DataFrame. Also, we should give these columns meaningful names.

In [25]:
# Drop unused columns and rename interesting columns.

# Keep interesting columns and rename them to something meaningful

# Mapping column index to name.
columnNames = {0: "id", 1: "name", 12: "results", 13: "violations"}
    
# Rename column from '_c{id}' to something meaningful.
cols = [inspections[i].alias(columnNames[i]) for i in columnNames.keys()]
   
# Drop columns we are not using.
df = inspections.select(cols).where(col('violations').isNotNull())

df.cache()
df.show()
df.count()

+------+--------------------+------------------+--------------------+
|    id|                name|           results|          violations|
+------+--------------------+------------------+--------------------+
|413707|       LUNA PARK INC|              Fail|24. DISH WASHING ...|
|391234|       CAFE SELMARIE|              Fail|2. FACILITIES TO ...|
|413751|          MANCHU WOK|              Pass|33. FOOD AND NON-...|
|120580|          SUSHI PINK|              Pass|32. FOOD AND NON-...|
|413715|              NABO'S|              Pass|19. OUTSIDE GARBA...|
|420207|  WHOLE FOODS MARKET|              Pass|32. FOOD AND NON-...|
|154514|         LAS FUENTES|              Fail|18. NO EVIDENCE O...|
|413711|CASA CENTRAL COMM...|              Pass|41. PREMISES MAIN...|
|413764|LA BRUQUENA RESTA...|Pass w/ Conditions|4. SOURCE OF CROS...|
|413757|WHIZ KIDS NURSERY...|              Pass|35. WALLS, CEILIN...|
|154516|TACO & BURRITO PA...|              Pass|30. FOOD IN ORIGI...|
|413759|  MARISCOS E

10469

In [26]:
df.take(1)

[Row(id=413707, name='LUNA PARK INC', results='Fail', violations='24. DISH WASHING FACILITIES: PROPERLY DESIGNED, CONSTRUCTED, MAINTAINED, INSTALLED, LOCATED AND OPERATED - Comments: All dishwashing machines must be of a type that complies with all requirements of the plumbing section of the Municipal Code of Chicago and Rules and Regulation of the Board of Health. OBSEVERD THE 3 COMPARTMENT SINK BACKING UP INTO THE 1ST AND 2ND COMPARTMENT WITH CLEAR WATER AND SLOWLY DRAINING OUT. INST NEED HAVE IT REPAIR. CITATION ISSUED, SERIOUS VIOLATION 7-38-030 H000062369-10 COURT DATE 10-28-10 TIME 1 P.M. ROOM 107 400 W. SURPERIOR. | 36. LIGHTING: REQUIRED MINIMUM FOOT-CANDLES OF LIGHT PROVIDED, FIXTURES SHIELDED - Comments: Shielding to protect against broken glass falling into food shall be provided for all artificial lighting sources in preparation, service, and display facilities. LIGHT SHIELD ARE MISSING UNDER HOOD OF  COOKING EQUIPMENT AND NEED TO REPLACE LIGHT UNDER UNIT. 4 LIGHTS ARE OUT 

The output of the above cell gives us an idea of the schema of the input file; the file includes the name of every establishment, the type of establishment, the address, the data of the inspections, and the location, among other things.

Let's start to get a sense of what our dataset contains. For example, what are the different values in the `results` column?

In [27]:
df.select('results').distinct().show()

+------------------+
|           results|
+------------------+
|              Fail|
|Pass w/ Conditions|
|              Pass|
+------------------+



In [28]:
df.groupBy('results').count().show()

+------------------+-----+
|           results|count|
+------------------+-----+
|              Fail| 2607|
|Pass w/ Conditions| 1028|
|              Pass| 6834|
+------------------+-----+



Let us develop a model that can guess the outcome of a food inspection, given the violations. Since logistic regression is a binary classification method, it makes sense to group our data into two categories: **Fail** and **Pass**. A "Pass w/ Conditions" is still a Pass, so when we train the model, we will consider the two results equivalent.

Let us go ahead and convert our existing dataframe(`df`) into a new dataframe where each inspection is represented as a label-violations pair. In our case, a label of 0 represents a failure, a label of 1 represents a success.

In [29]:
# The function to clean the data

labeledData = df.select(when(df.results == 'Fail', 0)
                        .when(df.results == 'Pass', 1)
                        .otherwise(1)
                        .alias('label'), 
                        'violations')
labeledData.show()

+-----+--------------------+
|label|          violations|
+-----+--------------------+
|    0|24. DISH WASHING ...|
|    0|2. FACILITIES TO ...|
|    1|33. FOOD AND NON-...|
|    1|32. FOOD AND NON-...|
|    1|19. OUTSIDE GARBA...|
|    1|32. FOOD AND NON-...|
|    0|18. NO EVIDENCE O...|
|    1|41. PREMISES MAIN...|
|    1|4. SOURCE OF CROS...|
|    1|35. WALLS, CEILIN...|
|    1|30. FOOD IN ORIGI...|
|    1|18. NO EVIDENCE O...|
|    1|18. NO EVIDENCE O...|
|    1|30. FOOD IN ORIGI...|
|    1|40. REFRIGERATION...|
|    1|38. VENTILATION: ...|
|    1|34. FLOORS: CONST...|
|    1|33. FOOD AND NON-...|
|    0|18. NO EVIDENCE O...|
|    1|33. FOOD AND NON-...|
+-----+--------------------+
only showing top 20 rows



Train a logistic regression model from the input dataframe

In [30]:
trainingData, testData = labeledData.randomSplit([0.8, 0.2])

tokenizer = Tokenizer(inputCol="violations", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features") 
# HashingTF的功能是将tokenizer转换为features向量
lr = LogisticRegression(maxIter=10, regParam=0.01)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

model = pipeline.fit(trainingData)

predictionsDf = model.transform(testData)
predictionsDf.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|label|          violations|               words|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|    0|1. SOURCE SOUND C...|[1., source, soun...|(262144,[1972,278...|[1.48422506159956...|[0.81520990329150...|       0.0|
|    0|1. SOURCE SOUND C...|[1., source, soun...|(262144,[4089,963...|[6.85827341414378...|[0.99895037633049...|       0.0|
|    0|10. SEWAGE AND WA...|[10., sewage, and...|(262144,[4640,487...|[10.2233134272921...|[0.99996368754419...|       0.0|
|    0|10. SEWAGE AND WA...|[10., sewage, and...|(262144,[2786,306...|[4.58834123777139...|[0.98993266835200...|       0.0|
|    0|10. SEWAGE AND WA...|[10., sewage, and...|(262144,[1972,269...|[9.73410913028309...|[0.99994077509174...|       0.0|
|    0|1

In [31]:
numSuccesses = predictionsDf.where('label == prediction').count()
numInspections = predictionsDf.count()

print ("There were %d inspections and there were %d successful predictions" % (numInspections, numSuccesses))
print("This is a %d%% success rate" % (float(numSuccesses) / float(numInspections) * 100))

There were 2134 inspections and there were 1868 successful predictions
This is a 87% success rate


### Cross-Validation（交叉验证）

CrossValidator begins by splitting the dataset into a set of folds which are used as separate training and test datasets. E.g., with k=5 folds, CrossValidator will generate 5 (training, test) dataset pairs, each of which uses 4/5 of the data for training and 1/5 for testing. To evaluate a particular ParamMap, CrossValidator computes the average evaluation metric for the 5 Models produced by fitting the Estimator on the 5 different (training, test) dataset pairs.

After identifying the best ParamMap, CrossValidator finally re-fits the Estimator using the best ParamMap and the entire dataset.

In [33]:
# We now treat the Pipeline as an Estimator, wrapping it in a CrossValidator instance.
# This will allow us to jointly choose parameters for all Pipeline stages.
# A CrossValidator requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
# We use a ParamGridBuilder to construct a grid of parameters to search over.
# With 3 values for hashingTF.numFeatures and 2 values for lr.regParam,
# this grid will have 3 x 2 = 6 parameter settings for CrossValidator to choose from.

paramGrid = ParamGridBuilder() \
    .addGrid(hashingTF.numFeatures, [10, 100, 1000]) \ 
    .addGrid(lr.regParam, [0.1, 0.01]) \        
    .build()

# 有三个备选项
# 有两个备选项
# crossval会自动尝试所有的combination of parameters

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=5)  

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(trainingData)

predictionsDf = cvModel.transform(testData)

numSuccesses = predictionsDf.where('label == prediction').count()
numInspections = predictionsDf.count()

print ("There were %d inspections and there were %d successful predictions" % (numInspections, numSuccesses))
print("This is a %d%% success rate" % (float(numSuccesses) / float(numInspections) * 100))

SyntaxError: unexpected character after line continuation character (<ipython-input-33-cf5e8db38af9>, line 9)

In [22]:
cvModel.explainParams()

estimator: estimator to be cross-validated (current: Pipeline_4d508b7dc95479bf7897)
estimatorParamMaps: estimator param maps (current: [{Param(parent=u'LogisticRegression_4de3adfa7c180a25b4d2', name='regParam', doc='regularization parameter (>= 0).'): 0.1, Param(parent=u'HashingTF_4bf9b60fc08483f17273', name='numFeatures', doc='number of features.'): 10}, {Param(parent=u'LogisticRegression_4de3adfa7c180a25b4d2', name='regParam', doc='regularization parameter (>= 0).'): 0.1, Param(parent=u'HashingTF_4bf9b60fc08483f17273', name='numFeatures', doc='number of features.'): 100}, {Param(parent=u'LogisticRegression_4de3adfa7c180a25b4d2', name='regParam', doc='regularization parameter (>= 0).'): 0.1, Param(parent=u'HashingTF_4bf9b60fc08483f17273', name='numFeatures', doc='number of features.'): 1000}, {Param(parent=u'LogisticRegression_4de3adfa7c180a25b4d2', name='regParam', doc='regularization parameter (>= 0).'): 0.01, Param(parent=u'HashingTF_4bf9b60fc08483f17273', name='numFeatures', doc='